# Component 8: Train DenseNet12150 epochs, ModelCheckpoint + ReduceLROnPlateau, **NO EarlyStopping**

In [ ]:
import tensorflow as tfimport pandas as pdimport numpy as npimport matplotlib.pyplot as pltimport jsonimport osfrom sklearn.utils.class_weight import compute_class_weightSEED = 42tf.random.set_seed(SEED)np.random.seed(SEED)OUTPUT_DIR = '../outputs'os.makedirs(f'{OUTPUT_DIR}/models', exist_ok=True)os.makedirs(f'{OUTPUT_DIR}/training_history', exist_ok=True)print('✓ Setup complete')

## Load Data

In [ ]:
train_df = pd.read_csv('../outputs/train_manifest.csv')val_df = pd.read_csv('../outputs/val_manifest.csv')IMG_SIZE = (224, 224)BATCH_SIZE = 32EPOCHS = 50NUM_CLASSES = len(train_df['class_label'].unique())print(f'Train: {{len(train_df)}} images')print(f'Val:   {{len(val_df)}} images')print(f'Classes: {{NUM_CLASSES}}')

## Preprocessing & Augmentation

In [ ]:
def preprocess(filepath, label):    img = tf.io.read_file(filepath)    img = tf.image.decode_jpeg(img, channels=3)    img = tf.image.resize(img, IMG_SIZE)    img = tf.keras.applications.densenet121.preprocess_input(img)    return img, labelaugmentation = tf.keras.Sequential([    tf.keras.layers.RandomFlip('horizontal'),    tf.keras.layers.RandomRotation(0.2)])def build_dataset(df, augment=False, shuffle=True):    ds = tf.data.Dataset.from_tensor_slices((df['filepath'].values, df['class_label'].values))    ds = ds.map(preprocess, tf.data.AUTOTUNE).cache()    if augment:        ds = ds.map(lambda x, y: (augmentation(x, training=True), y))    if shuffle:        ds = ds.shuffle(1000, seed=SEED)    return ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)train_ds = build_dataset(train_df, augment=True, shuffle=True)val_ds = build_dataset(val_df, augment=False, shuffle=False)print('✓ Datasets created')

## Build DenseNet121 Model

In [ ]:
base_model = tf.keras.applications.DenseNet121(    include_top=False,    pooling='avg',    weights='imagenet',    input_shape=(*IMG_SIZE, 3))# Freeze most layersfor layer in base_model.layers[:-20]:    layer.trainable = Falsemodel = tf.keras.Sequential([    base_model,    tf.keras.layers.Dense(128, activation='relu'),    tf.keras.layers.Dropout(0.5),    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')])model.summary()

## Compile & Setup

In [ ]:
model.compile(    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),    loss='sparse_categorical_crossentropy',    metrics=['accuracy'])# Compute class weightsclass_weights = compute_class_weight(    'balanced',    classes=np.unique(train_df['class_label']),    y=train_df['class_label'])class_weight_dict = {i: w for i, w in enumerate(class_weights)}print('Class weights:', class_weight_dict)

## Callbacks (NO EarlyStopping!)

In [ ]:
# ModelCheckpoint - saves best modelcheckpoint = tf.keras.callbacks.ModelCheckpoint(    f'{OUTPUT_DIR}/models/densenet121_best.h5',    monitor='val_accuracy',    save_best_only=True,    mode='max',    verbose=1)# ReduceLROnPlateau - reduces learning rate when stuckreduce_lr = tf.keras.callbacks.ReduceLROnPlateau(    monitor='val_loss',    factor=0.5,    patience=3,    min_lr=1e-7,    verbose=1)callbacks = [checkpoint, reduce_lr]print('✓ Callbacks configured (NO EarlyStopping)')

## Train Model

In [ ]:
print(f'\nStarting training for {EPOCHS} epochs...')print('='*60)history = model.fit(    train_ds,    validation_data=val_ds,    epochs=EPOCHS,    callbacks=callbacks,    class_weight=class_weight_dict,    verbose=1)print('\n' + '='*60)print('✅ TRAINING COMPLETE')print('='*60)

## Save History & Plots

In [ ]:
# Save history as JSON and CSVhistory_dict = history.historyhistory_path = f'{OUTPUT_DIR}/training_history/densenet121_history.json'csv_path = f'{OUTPUT_DIR}/training_history/densenet121_history.csv'with open(history_path, 'w') as f:    json.dump(history_dict, f, indent=2)import pandas as pdpd.DataFrame(history_dict).to_csv(csv_path, index=False)print(f'✓ History saved to {history_path}')print(f'✓ History saved to {csv_path}')# Plot training curvesfig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))# Accuracyax1.plot(history_dict['accuracy'], label='Train', linewidth=2)ax1.plot(history_dict['val_accuracy'], label='Validation', linewidth=2)ax1.set_title('Model Accuracy', fontsize=14, fontweight='bold')ax1.set_xlabel('Epoch')ax1.set_ylabel('Accuracy')ax1.legend()ax1.grid(alpha=0.3)# Lossax2.plot(history_dict['loss'], label='Train', linewidth=2)ax2.plot(history_dict['val_loss'], label='Validation', linewidth=2)ax2.set_title('Model Loss', fontsize=14, fontweight='bold')ax2.set_xlabel('Epoch')ax2.set_ylabel('Loss')ax2.legend()ax2.grid(alpha=0.3)plt.suptitle(f'DenseNet121 Training Progress', fontsize=16, fontweight='bold')plt.tight_layout()plot_path = f'{OUTPUT_DIR}/training_history/densenet121_curves.png'plt.savefig(plot_path, dpi=200, bbox_inches='tight')plt.show()print(f'✓ Training curves saved to {plot_path}')print(f'\nBest model saved to: {OUTPUT_DIR}/models/densenet121_best.h5')print(f'Best val_accuracy: {max(history_dict["val_accuracy"]):.4f}')